### Takes my smorf dataset and uses NeuroSEED's code to store it in the same way

In [10]:
# Import code

import sys
sys.path.insert(0,'../')

from scipy.stats import mode
import argparse
import pickle
import pandas as pd

from edit_distance.task.dataset_generator_genomic import EditDistanceGenomicDatasetGenerator

In [11]:
# save dataset as string
df = pd.read_csv("../datasets/dataset_FINAL.tsv", sep='\t')

file_out='../datasets/strings.txt'

no_weird_chars=df['smorf'].str.contains(r'^[ACTG]+$', na=False)

with open(file_out, 'w') as f_out:
    f_out.writelines("%s\n" % l for l in df[no_weird_chars].smorf.values)



In [47]:
# create string to be read by NeuroSEED-borrowed code
file_out='../datasets/strings.txt'

parser = argparse.ArgumentParser()
parser.add_argument('--out', type=str, default="../datasets/strings.pkl", help='Output data path')
parser.add_argument('--train_size', type=int, default=20000, help='Training sequences')
parser.add_argument('--val_size', type=int, default=2000, help='Validation sequences')
parser.add_argument('--test_size', type=int, default=4000, help='Test sequences')
parser.add_argument('--source_sequences', type=str, default=file_out, help='Sequences data path')
args, unknown = parser.parse_known_args()

# load and divide sequences
with open(args.source_sequences, 'rb') as f:
    L = f.readlines()
L = [l[:-1].decode('UTF-8') for l in L]

strings = {
    'train': L[:args.train_size],
    'val': L[args.train_size:args.train_size + args.val_size],
    'test': L[args.train_size + args.val_size:args.train_size + args.val_size + args.test_size]
}

data = EditDistanceGenomicDatasetGenerator(strings=strings)
data.save_as_pickle(args.out)




Generating train


Edit distance 7000x7000:  55%|█████▍    | 3848/7000 [02:24<01:53, 27.88it/s]

In [3]:
with open('../datasets/string_subset.pkl', 'rb') as f:
        sequences, distances = pickle.load(f)

print(distances.keys())
slices={'train':100, 'test':10, 'val':20}
smaller_distances = {key: distances[key][:slices[key],:slices[key]] for key in distances.keys()}
smaller_sequences = {key: sequences[key][:slices[key]] for key in sequences.keys()}

pickle.dump((smaller_sequences, smaller_distances),open('../datasets/string_for_test' + ".pkl", "wb"))

dict_keys(['train', 'val', 'test'])


In [9]:
# %%

df = pd.read_csv("../datasets/dataset_FINAL.tsv", sep='\t')
smorfams=df.clust[df.clust.str.startswith('smorfam') & df.y.str.fullmatch('positive')]
md,count=mode(smorfams)
no_weird_chars=df['smorf'].str.contains(r'^[ACTG]+$', na=False)

to_write=df.clust.str.startswith(md[0]) & df.y.str.fullmatch('positive') & no_weird_chars

file_out='../datasets/largest_group_strings.txt'

with open(file_out, 'w') as f_out:
    f_out.writelines("%s\n" % l for l in df[to_write].smorf.values)

parser = argparse.ArgumentParser()
parser.add_argument('--out', type=str, default="../datasets/largest_group_strings.pkl", help='Output data path')
parser.add_argument('--train_size', type=int, default=50, help='Training sequences')
parser.add_argument('--val_size', type=int, default=7, help='Validation sequences')
parser.add_argument('--test_size', type=int, default=7, help='Test sequences')
parser.add_argument('--source_sequences', type=str, default=file_out, help='Sequences data path')
args, unknown = parser.parse_known_args()

# load and divide sequences
with open(args.source_sequences, 'rb') as f:
    L = f.readlines()
L = [l[:-1].decode('UTF-8') for l in L]

strings = {
    'train': L[:args.train_size],
    'val': L[args.train_size:args.train_size + args.val_size],
    'test': L[args.train_size + args.val_size:args.train_size + args.val_size + args.test_size]
}

data = EditDistanceGenomicDatasetGenerator(strings=strings)
data.save_as_pickle(args.out)

Generating train


Edit distance 50x50: 100%|██████████| 50/50 [00:00<00:00, 2955.69it/s]

Time to compute the matrix: 0.02004694938659668
Generating val



Edit distance 7x7: 100%|██████████| 7/7 [00:00<00:00, 12504.31it/s]

Time to compute the matrix: 0.003579378128051758
Generating test



Edit distance 4x4: 100%|██████████| 4/4 [00:00<00:00, 82646.38it/s]


Time to compute the matrix: 0.0031969547271728516
